fix this notebook link<!-- <a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/ssssxxxx.ipynb" 
    target="_blank"
    >
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a> -->

# Create a Custom Tapis App  📒
***Create your own Tapis App***

by Silvia Mazzoni, DesignSafe, 2025

This is a step-by-step guide to help you write your own **Tapis v3 App** — from defining the app, to registering it, and running it. This is a practical walkthrough for defining and deploying an **HPC or VM-based** app using the Tapis v3 API.

In [1]:
pwd

'/home/jupyter/MyData/OpenSees/OpenSeesPy_TapisApp'

In [2]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    print('using communitydata')
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

using communitydata


In [3]:
import json

## Connect to Tapis

In [4]:
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2025-08-17T09:12:28+00:00
 Token expires in: 3:01:11.266899
-- LOG IN SUCCESSFUL! --


In [5]:
app_id = 'opensees-mp-s3-silvia-new'

### Get username

In [6]:
user_info = t.authenticator.get_userinfo()
username = user_info.username

## Get latest version of the app

In [7]:
latest = t.apps.getAppLatestVersion(appId=app_id)
print(latest)


containerImage: tapis://cloud.data/corral/tacc/aci/CEP/applications/v3/opensees/latest/OpenSees/opensees.zip
created: 2025-08-16T18:53:58.422294Z
deleted: False
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
enabled: True
id: opensees-mp-s3-silvia-new
isPublic: False
jobAttributes: 
archiveOnAppError: True
archiveSystemDir: HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}
archiveSystemId: stampede3
cmdPrefix: None
coresPerNode: 48
description: None
dtnSystemInputDir: !tapis_not_set
dtnSystemOutputDir: !tapis_not_set
dynamicExecSystem: False
execSystemConstraints: None
execSystemExecDir: ${JobWorkingDir}
execSystemId: stampede3
execSystemInputDir: ${JobWorkingDir}
execSystemLogicalQueue: skx
execSystemOutputDir: ${JobWorkingDir}
fileInputArrays: []
fileInputs: [
autoMountLocal: True
description: Input directory that includes the tcl script as well as any other requ

In [8]:

# def resolve_latest_app_version(t, app_id: str) -> str:
#     """
#     Return the latest enabled app version for `app_id`.
#     Falls back to manual SemVer sort if the 'latest' endpoint isn't available.

#     Author
#     ------
#     Silvia Mazzoni, DesignSafe (silviamazzoni@yahoo.com)

#     Date
#     ----
#     2025-08-14

#     Version
#     -------
#     1.0
#     """
#     # from packaging.version import Version
#     # 1) Try the official helper (if present in your Tapipy version)
#     try:
#         latest = t.apps.getAppLatestVersion(appId=app_id)
#         ver = getattr(latest, "version", None) or (latest.get("version") if isinstance(latest, dict) else None)
#         if ver:
#             return ver
#     except Exception:
#         print(f'Unable to find latest version of app: {app_id}')
#         return 'none'



In [9]:


# def resolve_latest_app_version(
#     t,
#     app_id: str,
#     allow_literal_latest_if_only: bool = True,
#     allow_literal_latest_if_newest: bool = False,
# ) -> str:
#     """
#     Resolve a concrete app version for `app_id`, handling Tapipy resources vs dicts,
#     and the presence of a literal version named "latest".

#     Resolution order (defaults favor reproducibility):
#       1) Try Apps helper getAppLatestVersion:
#          - If it returns a concrete version (not "latest"), return it.
#          - If it returns "latest", enumerate all versions (step 2).
#       2) Enumerate enabled versions via getApps(search=...):
#          - If "latest" is the ONLY enabled version → return "latest"
#            (controlled by allow_literal_latest_if_only; default True).
#          - Else pick highest semantic version (stable > prerelease).
#          - If none parseable and `allow_literal_latest_if_newest` is True and the newest
#            record is the literal "latest", return "latest".
#       3) If no SemVer at all but a literal "latest" exists → return "latest".
#       4) Otherwise → return "none".

#     Parameters
#     ----------
#     t : tapipy.tapis.Tapis
#     app_id : str
#     allow_literal_latest_if_only : bool, default True
#     allow_literal_latest_if_newest : bool, default False

#     Returns
#     -------
#     str
#         Concrete version like "1.2.3", or "latest" (per flags/last resort), or "none".

#     Author
#     ------
#     Silvia Mazzoni, DesignSafe (silviamazzoni@yahoo.com)
#     Date: 2025-08-16, Version: 1.4
#     """
#     from typing import Any, List, Tuple
#     from packaging.version import Version
#     def _field(obj: Any, name: str, default=None):
#         # Works for Tapipy Resource objects and dicts
#         if isinstance(obj, dict):
#             return obj.get(name, default)
#         return getattr(obj, name, default)

#     # --- 1) Try official "latest" helper
#     try:
#         latest = t.apps.getAppLatestVersion(appId=app_id)
#         latest = getattr(latest, "result", latest)
#         v = _field(latest, "version")
#         if isinstance(v, str) and v:
#             if v.lower() != "latest":
#                 return v
#             # if literal "latest", continue to enumeration for a reproducible SemVer
#     except Exception:
#         pass

#     # --- 2) Enumerate all enabled versions
#     try:
#         resp = t.apps.getApps(search=f"(id.eq.{app_id})",
#                               listType="ALL",
#                               select="id,version,enabled,versionEnabled,created")
#         items = getattr(resp, "result", resp)
#         if not isinstance(items, list):
#             items = [items] if items else []

#         # Filter enabled (either flag may be present depending on endpoint)
#         enabled: List[Any] = []
#         for it in items:
#             en = _field(it, "enabled", True)
#             ven = _field(it, "versionEnabled", True)
#             if bool(en) and bool(ven):
#                 enabled.append(it)

#         if not enabled:
#             return "none"

#         # Only one enabled?
#         if len(enabled) == 1:
#             v = _field(enabled[0], "version", "") or ""
#             v = v.strip()
#             if v.lower() == "latest":
#                 return "latest" if allow_literal_latest_if_only else "none"
#             return v if v else "none"

#         # Multiple enabled → collect metadata
#         def created_key(it):
#             return _field(it, "created", "") or ""

#         # Does a literal latest exist? Is it newest?
#         literal_latest_records = [it for it in enabled
#                                   if isinstance(_field(it, "version", ""), str)
#                                   and _field(it, "version", "").lower() == "latest"]
#         has_literal_latest = len(literal_latest_records) > 0
#         newest = max(enabled, key=created_key) if enabled else None
#         newest_is_literal_latest = bool(
#             newest and isinstance(_field(newest, "version", ""), str) and _field(newest, "version", "").lower() == "latest"
#         )

#         # Build SemVer sets
#         stable: List[Tuple[Version, str]] = []
#         prerelease: List[Tuple[Version, str]] = []
#         for it in enabled:
#             v = (_field(it, "version", "") or "").strip()
#             if not v or v.lower() == "latest":
#                 continue
#             try:
#                 sv = Version(v)
#                 (stable if not sv.is_prerelease else prerelease).append((sv, v))
#             except Exception:
#                 # skip non-SemVer (e.g., "dev", "abc")
#                 continue

#         if stable:
#             stable.sort(key=lambda x: x[0])
#             return stable[-1][1]
#         if prerelease:
#             prerelease.sort(key=lambda x: x[0])
#             return prerelease[-1][1]

#         # No SemVer at all
#         if has_literal_latest:
#             if newest_is_literal_latest and allow_literal_latest_if_newest:
#                 return "latest"
#             return "latest"  # last resort

#         return "none"

#     except Exception:
#         return "none"


In [10]:
latest_app_version = OpsUtils.get_latest_app_version(t,app_id)
print('latest_app_version:',latest_app_version)

AttributeError: module 'OpsUtils.OpsUtils' has no attribute 'get_latest_app_version'

### All apps

In [ ]:
listType = 'ALL' # Include all items requester is authorized to view. Includes check for READ or MODIFY permission.
select = 'id,created,description,version,owner' # Attributes to return in each result.
orderBy = 'created(asc)'
results = t.apps.getApps( orderBy=orderBy,
                         select=select)  
for thisRes in results:
    print('--')
    print(thisRes)

### the new app

In [ ]:
t.apps.getAppLatestVersion(appId=app_def["id"])

## ✅ Step 5: Submit a Job

You’ll now submit a job using this app. You can use the Tapis CLI, Tapipy, or a web form.


In [ ]:
print('app_name:',app_name)
print('latest_app_version:',latest_app_version)

In [ ]:
a = 3/0

### App-Specific Input

In [ ]:
# initalize
tapisInputAll = {}
tapisInputAll["name"] = 'SilviaApp_OpenSeesMP'

tapisInputAll["appId"] = app_name # options: "opensees-express", "opensees-mp-s3", "opensees-2p-s3"
tapisInputAll["appVersion"] = app_version # always use latest in this Notebook Template
tapisInputAll["maxMinutes"] = 6

# OpenSees-mp-s3 and OpenSees-xp-s3 only:
tapisInputAll["execSystemId"] = "stampede3" # the app runs on stampede only
tapisInputAll["execSystemLogicalQueue"] = "skx-dev" # "skx", "skx-dev"
tapisInputAll["nodeCount"] = 1 # limits set by which compute nodes you use
tapisInputAll["coresPerNode"] = 48 # limits set by which compute nodes you use
tapisInputAll["allocation"] = "DS-HPC1"

tapisInputAll['archive_system']='MyData' # Options: MyData or Work

### Job-Specific Input

In [ ]:
# initalize
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData' # ########################## unique

tapisInput["name"] = tapisInput["name"] + '_' + tapisInput['storage_system']

tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'  # ########################## unique
tapisInput['input_filename'] = 'Ex1a_verymany.Canti2D.Push.mp.tcl' # ########################## unique



## Submit Job

In [ ]:
# -----------------------------------------------------
jobReturns = OpsUtils.run_tapis_job(t,tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------

In [ ]:
a = 3/0

In [ ]:
tapisInput_tmp = tapisInput.copy()
tapisInput_tmp['appId'] = 'opensees-mp-s3'
tapisInput_tmp['appVersion'] = 'latest'
print('tapisInput_tmp',tapisInput_tmp)

job_description = OpsUtils.get_tapis_job_description(t, tapisInput_tmp)

print('job_description',job_description)

job_description['appId'] = app_name
# job_description['appVersion'] = app_version
print('job_description',job_description)
    # returnDict = OpsUtils.submit_tapis_job(t, job_description, askConfirmJob)


In [ ]:
askConfirmJob = False
returnDict = OpsUtils.submit_tapis_job(t, job_description, askConfirmJob)

In [ ]:
jobUuid = returnDict.get("jobUuid")
askConfirmMonitorRT = False
OpsUtils.monitor_tapis_job(t, jobUuid, returnDict.get("job_start_time"), askConfirmMonitorRT)

# Always fetch basic status after monitoring
JobStatusData = OpsUtils.get_tapis_job_status(t, jobUuid, tapisInput)

# Optional enrichments
JobMetadata = OpsUtils.get_tapis_job_metadata(t, jobUuid, tapisInput)
JobHistory = OpsUtils.get_tapis_job_history_data(t, jobUuid)
JobFiledata = OpsUtils.get_tapis_job_all_files(t, jobUuid)

## ✅ Summary

| Step                        | Description                                                         |
| --------------------------- | ------------------------------------------------------------------- |
| 1. Create `app.json`        | Describes the app, its inputs, execution system, and wrapper script |
| 2. Create `tapisjob_app.sh` | Runs your analysis (e.g., ibrun OpenSees main.tcl)                  |
| 3. Create `profile.json`    | (Optional) Loads modules/environment                                |
| 4. Upload Files             | To the deployment path in your storage system                       |
| 5. Register App             | With Tapis via CLI or Python                                        |
| 6. Submit Job               | Define `job.json` and submit                                        |



Awesome—here are concise, ready-to-paste **best-practice notes** to follow your Step 3.

# ✅ Best Practices for Tapis v3 App Deployment

## 1) Directory & Versioning Strategy

* **One app, many versions:**

  * `apps/<app_id>/<version>/` (e.g., `apps/opensees-mp/1.0/`)
  * Treat each `<version>` as **immutable** once registered.
* **Use semantic versions:** `1.0.0`, `1.0.1`, `1.1.0` (avoid “latest” in the path).
* **Optional convenience alias:** keep a lightweight `latest` folder that *copies* the newest version’s files (don’t register “latest” as the app version—register the semantic version).

## 2) Environments: dev → test → prod

* Keep **separate app IDs** or **separate versions** for lifecycle stages.

  * Example:

    * `opensees-mp-dev/0.9.x` (fast iteration; looser validation)
    * `opensees-mp-test/1.0.0-rc1` (candidate)
    * `opensees-mp/1.0.0` (production)
* Change only:

  * `executionSystem`, SLURM queue, default node/core limits
  * container image tag, module set in `profile.json`
* Keep **input schema** stable between test and prod to avoid breaking users.

## 3) Files to include per version

* `app.json` – App metadata, inputs/parameters, execution/deployment systems.
* `profile.json` – Environment setup for the execution system (modules, PATH, MPI launcher, etc.).
* `tapisjob_app.sh` – Wrapper that launches the job (container/module loads, cd, run, write exitcode).
* Optional:

  * `README.md` – What this app/version does, changes vs. prior version, known issues.
  * `CHANGELOG.md` – Human-readable diff for operators and users.
  * `examples/` – Minimal working inputs for quick validation.

## 4) Permissions & Ownership

* **Deployment system path should be read-only** to regular users after registration.
* Ensure **app owner** (and ops team) retains write access to `apps/<app_id>/`.
* Use a **service or team account** as `owner` in `app.json` where possible for continuity.

## 5) Registration & Immutability

* Upload files → **register** the app/version → **do not mutate** files in place.
* If you must change something, **create a new version** (`1.0.1`) and re-register.
* Keep a record of the **registered app UUID** and the git commit (if any) that produced it.

## 6) Validation Checklist (quick)

* `app.json` parses, required fields present (id, version, executionSystem, deploymentSystem, runtime).
* `profile.json` matches **executionSystem** (modules exist, `OpenSees*` path/command resolvable).
* `tapisjob_app.sh`:

  * `set -e` (or robust exit checks), write `${_tapisExecSystemOutputDir}/tapisjob.exitcode`
  * correct working dir & mounts, prints env + version for debug
* Submit a **smoke test** job using a tiny input in `examples/`:

  * Confirms launch, output capture, archive, and non-zero exit handling.

## 7) Consistent Inputs & Defaults

* In `app.json`, define **clear, minimal** required inputs/params.
* Provide safe **defaults** (walltime, nodes/cores, queue) for novice users.
* Document **Tcl vs. OpenSeesPy** expectations (e.g., `mainProgram`, `tclScript`, `pyScript`).

## 8) Separation of Concerns (why split files)

* `app.json` = **What** the app is and needs (schema, systems, resources).
* `profile.json` = **How** to prepare the environment on the **execution system** (modules, env vars).
* `tapisjob_app.sh` = **How** to run the job (container/module, mpirun/ibrun, error plumbing).
* This separation keeps **runtime environment tweaks** out of your stable app contract.

## 9) Container & Modules

* Prefer a **versioned container tag** for reproducibility (e.g., `taccaci/opensees:2025.06`).
* If using **modules**, pin specific versions in `profile.json` and echo them at runtime for provenance.

## 10) Provenance & Logging

* At job start, print: app id/version, container/tag, module list, SLURM env, date/time, hostname.
* Always write explicit **exit code** and a short **status summary** to stdout/stderr.




## look for my apps

In [ ]:
a = 3/0

In [ ]:
listType = 'ALL' # Include all items requester is authorized to view. Includes check for READ or MODIFY permission.
select = 'id,created,description,version,owner' # Attributes to return in each result.
orderBy = 'created(asc)'
results = t.apps.getApps( orderBy=orderBy,
                         select=select)  
for thisRes in results:
    print('--')
    print(thisRes)

In [ ]:
thisAppData = OpsUtils.get_tapis_app_schema(t,'shell-runner',version='1.0.0')
OpsUtils.display_tapis_app_schema(thisAppData)

In [ ]:
thisAppData = OpsUtils.get_tapis_app_schema(t,'opensees-mp-s3',version='latest')
OpsUtils.display_tapis_app_schema(thisAppData)

## look for all apps

In [ ]:
listType = 'ALL' # Include all items requester is authorized to view. Includes check for READ or MODIFY permission.
select = 'id,created,description,version,owner' # Attributes to return in each result.
orderBy = 'created(asc)'
results = t.apps.getApps( orderBy=orderBy,
                         listType=listType,
                         select=select)  
for thisRes in results:
    print('--')
    print(thisRes)

## Look for all apps with opensees in their id, and the latest version

In [ ]:
searchQuery = "(id.like.*opensees*)~(version.eq.latest)"
listType = 'ALL'
select = 'id,created,description,version'
orderBy = 'created(asc)'
results = t.apps.getApps(search=searchQuery,
                         orderBy=orderBy,
                         listType=listType,
                         select=select)  
for thisRes in results:
    print('--')
    print(thisRes)

## Look for more Specific app

In [ ]:
searchQuery = "(id.like.*opensees*mp*)~(version.eq.latest)"
results = t.apps.getApps(search=searchQuery,
                listType=listType,select=select)  
for thisRes in results:
    print('--')
    print(thisRes)

## Look for a different version

In [ ]:
searchQuery = "(id.like.*opensees*mp*)~(version.eq.3.5.0)"
results = t.apps.getApps(search=searchQuery,
                listType=listType,select=select)  
for thisRes in results:
    print('--')
    print(thisRes)
    

### Use a utility function:

In [ ]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils,['query_tapis_apps.py'])

In [ ]:
results = OpsUtils.query_tapis_apps(t,['opensees','mp'],version='latest',select = 'id,created,description,version')    
print(results)
print('------------')
results = OpsUtils.query_tapis_apps(t,['opensees','mp'],select = 'id,created,description,version')    
print(results)
print('------------')
results = OpsUtils.query_tapis_apps(t,['opensees','mp'],version='latest')    
print(results[0].id)